In [1]:
import random

class State:

    def __init__(self, prices=None, enemy_prices=None):
        self.prices = prices if prices != None else [0]*9
        self.enemy_prices = enemy_prices if enemy_prices != None else [0]*9

    def price_count(self, prices):
        count=0
        for i in prices:
            if i == 1:
                count += 1
            
        return count
    
    def is_lose(self):
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or self.enemy_prices[x + y * 3] == 0:
                    return False
                
                x = x + dx
                y = y + dy
            return True
        
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False
    
    def is_draw(self):
        return self.price_count(self.prices) + self.price_count(self.enemy_prices) == 9
    
    def is_done(self):
        return self.is_lose() or self.is_draw()
    
    def next(self, action):
        prices = self.prices.copy()
        prices[action] = 1
        return State(self.enemy_prices, prices)
    
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.prices[i] == 0 and self.enemy_prices[i] == 0:
                actions.append(i)
        return actions
    
    def is_first_player(self):
        return self.price_count(self.prices) == self.price_count(self.enemy_prices)
    
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str=''
        for i in range(9):
            if self.prices[i] == 1:
                str += ox[0]
            elif self.enemy_prices[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

In [2]:
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

In [3]:
state = State()

while True:

    if state.is_done():
        break

    action = random_action(state)

    state = state.next(action)

    print(state)

-o-
---
---

xo-
---
---

xoo
---
---

xoo
---
--x

xoo
---
-ox

xoo
---
xox

xoo
--o
xox

xoo
-xo
xox



In [4]:
def mini_max(state):
    if state.is_lose():
        return -1
    
    if state.is_draw():
        return 0
    
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_score = score

    return best_score

In [5]:
def mini_max_action(state):

    best_action = 0
    best_score = -float('inf')
    str = ['', '']
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_action = action
            best_score = score

        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
    print('action:', str[0], '\nscore: ', str[1], '\n')

    return best_action


In [6]:
state = State()

while True:

    if state.is_done():
        break

    if state.is_first_player():
        action = mini_max_action(state)
    else:
        action = random_action(state)

    state = state.next(action)

    print(state)

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

o--
---
---

o--
---
-x-

action:  1, 2, 3, 4, 5, 6, 8, 
score:   0, 1,-1, 1, 0, 1, 0, 

o-o
---
-x-

oxo
---
-x-

action:  3, 4, 5, 6, 8, 
score:  -1, 1,-1,-1,-1, 

oxo
-o-
-x-

oxo
xo-
-x-

action:  5, 6, 8, 
score:   1, 1, 1, 

oxo
xoo
-x-

oxo
xoo
-xx

action:  6, 
score:   1, 

oxo
xoo
oxx



In [7]:
def mini_max_plus(state, limit):

    if state.is_lose():
        return -1
    
    if state.is_draw():
        return 0
    
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max_plus(state.next(action), -best_score)
        if score > best_score:
            best_score = score

        if best_score >= limit:
            return best_score
        
    return best_score

In [11]:
def alpha_beta(state, alpha, beta):

    if state.is_lose():
        return -1
    
    if state.is_draw():
        return 0
    
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha)
        if score > alpha:
            alpha = score

        if alpha >= beta:
            return alpha
        
    return alpha
    

In [12]:
def alpha_beta_action(state):
    best_action = 0
    alpha = -float('inf')
    str = ['', '']
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -float('inf'), -alpha)
        if score > alpha:
            best_action = action
            alpha = score

        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
    print('action:', str[0], '\nscore: ', str[1], '\n')

    return best_action


In [14]:
state = State()

while True:

    if state.is_done():
        break

    if state.is_first_player():
        action = alpha_beta_action(state)
    else:
        action = mini_max_action(state)

    state = state.next(action)

    print(state)

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

o--
---
---

action:  1, 2, 3, 4, 5, 6, 7, 8, 
score:  -1,-1,-1, 0,-1,-1,-1,-1, 

o--
-x-
---

action:  1, 2, 3, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 

oo-
-x-
---

action:  2, 3, 5, 6, 7, 8, 
score:   0,-1,-1,-1,-1,-1, 

oox
-x-
---

action:  3, 5, 6, 7, 8, 
score:  -1,-1, 0, 0, 0, 

oox
-x-
o--

action:  3, 5, 7, 8, 
score:   0,-1,-1,-1, 

oox
xx-
o--

action:  5, 7, 8, 
score:   0,-1,-1, 

oox
xxo
o--

action:  7, 8, 
score:   0, 0, 

oox
xxo
ox-

action:  8, 
score:   0, 

oox
xxo
oxo



In [15]:
def playout(state):
    if state.is_lose():
        return -1
    
    if state.is_draw():
        return 0
    
    return -playout(state.next(random_action(state)))

In [16]:
def mcs_action(state):

    legal_actions = state.legal_actions()
    values = [0] * len(legal_actions)
    for i, action in enumerate(legal_actions):
        for _ in range(10):
            values[i] += -playout(state.next(action))

    return legal_actions[argmax(values)]

In [17]:
def argmax(collection, key = None):
    return collection.index(max(collection))

In [20]:
EP_GAME_COUNT = 100

def first_player_point(ended_state):

    if ended_state.is_lose():
        return 0 if ended_state.is_first_player() else 1
    return 0.5
def play(next_actions):
    state = State()

    while True:
        if state.is_done():
            break

        next_action = next_actions[0] if state.is_first_player() else next_actions[1]
        action = next_action(state)

        state = state.next(action)

    return first_player_point(state)

def evaludate_algorithm_of(label, next_actions):
    total_point = 0
    for i in range(EP_GAME_COUNT):
        if i % 2 == 0:
            total_point += play(next_actions)
        else:
            total_point += 1 - play(list(reversed(next_actions)))

        print('\rEvaluate {}/{}'.format(i+1, EP_GAME_COUNT), end='')
    print('')

    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

next_actions = (mcs_action, random_action)
evaludate_algorithm_of('VS_RANDOM {:3f}', next_actions)


next_actions = (mcs_action, alpha_beta_action)
evaludate_algorithm_of('VS_ALPHABETA {:3f}', next_actions)

Evaluate 100/100
VS_RANDOM 0.870000
action:  0, 1, 2, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 4, 5, 6, 8, 
score:  -1, 1, 1, 1, 1, 1, 

action:  4, 5, 6, 8, 
score:   1,-1,-1,-1, 

action:  5, 6, 
score:  -1, 1, 

Evaluate 1/100action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, 
score:  -1,-1, 0, 0, 0, 

action:  5, 7, 8, 
score:   0,-1,-1, 

action:  8, 
score:   0, 

Evaluate 2/100action:  1, 2, 3, 4, 5, 6, 7, 8, 
score:  -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 8, 
score:  -1,-1, 0, 0, 0, 0, 

action:  1, 5, 6, 8, 
score:   0, 1,-1,-1, 

Evaluate 3/100action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 5, 7, 8, 
score:   1, 1, 1, 1, 1, 1, 1, 

action:  3, 4, 5, 7, 8, 
score:  -1, 1, 1, 1, 1, 

action:  3, 5, 7, 
score:  -1,-1, 1, 

Evaluate 4/100action:  0, 1, 3, 4, 5, 6, 7, 8, 
score:  -1,-1,-1

In [26]:
import math

def mcts_action(state):
    class Node:
        def __init__(self, state):
            self.state = state
            self.w = 0
            self.n = 0
            self.child_nodes = None

        def eval(self):
            if self.state.is_done():
                value = -1 if self.state.is_lose() else 0

                self.w += value
                self.n += 1
                return value
            
            if not self.child_nodes:
                value = playout(self.state)

                self.w += value
                self.n += 1

                if self.n == 10:
                    self.expand()
                return value
            else:
                value = -self.next_child_node().eval()

                self.w += value
                self.n += 1
                return value
        
        def expand(self):
            legal_actions = self.state.legal_actions()
            self.child_nodes = []
            for action in legal_actions:
                self.child_nodes.append(Node(self.state.next(action)))

        def next_child_node(self):

            for child_node in self.child_nodes:
                if child_node.n == 0:
                    return child_node
                
            t = 0
            for c in self.child_nodes:
                t += c.n
            ucb1_values = []
            for child_node in self.child_nodes:
                ucb1_values.append(-child_node.w / child_node.n + (2 * math.log(t) / child_node.n) ** .5)

            return self.child_nodes[argmax(ucb1_values)]

    root_node = Node(state)
    root_node.expand()

    for _ in range(100):
        root_node.eval()

    legal_actions = state.legal_actions()
    n_list = []
    for c in root_node.child_nodes:
        n_list.append(c.n)

    return legal_actions[argmax(n_list)]

In [27]:
EP_GAME_COUNT = 100

def first_player_point(ended_state):

    if ended_state.is_lose():
        return 0 if ended_state.is_first_player() else 1
    return 0.5
def play(next_actions):
    state = State()

    while True:
        if state.is_done():
            break

        next_action = next_actions[0] if state.is_first_player() else next_actions[1]
        action = next_action(state)

        state = state.next(action)

    return first_player_point(state)

def evaludate_algorithm_of(label, next_actions):
    total_point = 0
    for i in range(EP_GAME_COUNT):
        if i % 2 == 0:
            total_point += play(next_actions)
        else:
            total_point += 1 - play(list(reversed(next_actions)))

        print('\rEvaluate {}/{}'.format(i+1, EP_GAME_COUNT), end='')
    print('')

    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

next_actions = (mcts_action, random_action)
evaludate_algorithm_of('VS_RANDOM {:3f}', next_actions)


next_actions = (mcts_action, alpha_beta_action)
evaludate_algorithm_of('VS_ALPHABETA {:3f}', next_actions)

Evaluate 100/100
VS_RANDOM 0.045000
action:  0, 1, 2, 3, 4, 6, 7, 8, 
score:  -1,-1, 0, 0, 0, 0, 0, 0, 

action:  0, 1, 3, 4, 6, 8, 
score:   1, 1, 1, 1, 1, 1, 

action:  1, 4, 6, 8, 
score:   1, 1, 1, 1, 

Evaluate 1/100action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 6, 7, 8, 
score:  -1, 1, 1, 1, 1, 1, 1, 

action:  1, 3, 4, 6, 7, 
score:   1, 1, 1, 1, 1, 

Evaluate 2/100action:  1, 2, 3, 4, 5, 6, 7, 8, 
score:  -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 
score:   0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 
score:  -1, 1, 1, 1, 

action:  3, 7, 
score:   1, 1, 

Evaluate 3/100action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  2, 3, 4, 5, 6, 7, 8, 
score:   0, 1, 1, 1, 1, 1, 1, 

action:  2, 4, 6, 7, 8, 
score:   0, 1, 1, 1, 1, 

action:  2, 6, 8, 
score:   1, 1, 1, 

action:  8, 
score:   1, 

Evaluate 4/100action:  0, 1, 2, 3, 4, 5, 6, 8, 
score:  -1, 0, 0, 0, 0, 0, 0, 0, 

action:  0, 3, 4, 5,